<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/eda/AI_risks_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets

* AI Incident Database
https://www.kaggle.com/datasets/konradb/ai-incident-database
* Algorithmic Justice League's Biased Algorithm Datasets
* Government AI Readiness Index
* AI Ethics Guidelines Global Inventory
* Data & Society Reports
* AI Now Institute Reports

# Questions

1. What are main AI failure types?
2. Failure types evolution through time.
3. Failure types by domain/company/country/demographics/models.
4. Consequences of AI failures
5. How these AI risks map onto existing ethical guidelines and regulations?
6. What are the mitigation strategies that are most effective?
7. How to prevent those risks?

